# Desrcibe LTCM's investment strategy

1) The securities traded include government bonds, interest rate swaps, mortgage-pools, FRAs, options and stocks. Their trading frequency was relatively low targeting a longer-term, low turnover approach allowing their positions to converge. They sought after small consistent gains as opposed to infrequent big-hitters. The firms trade selection was driven by a view on spreads between various instruments, determined by quantiative models based on historical data and a view of markets going forward.


## What are LTCM's biggest advantages over their competitors

2. Expertise and finanical technology set gave LTCM an edge over their competitors, as well as their ability to obtain favouable financing terms. 

## LTCM Funding Risk

3. Collateral haircuts: LTCM 

In [229]:
import pandas as pd 
import numpy as np 
import portfolio_management_helper as pmh 

ltcm_infile = "../data/ltcm_exhibits_data.xlsx"
gmo_infile = "../data/gmo_analysis_data.xlsx"

ltcm_check = pd.read_excel(ltcm_infile, sheet_name=None)
gmo_check = pd.read_excel(gmo_infile, sheet_name=None)

ltcm_sheets = list(ltcm_check.keys())
gmo_sheets = list(gmo_check.keys())

ltcm_performance = pmh.read_excel_default(ltcm_infile, parse_dates=True, index_col=0, sheet_name=ltcm_sheets[1])[:-4]
ltcm_balance = pmh.read_excel_default(ltcm_infile, parse_dates=True, index_col=0, sheet_name=ltcm_sheets[2])[:-2]
risk_free = pmh.read_excel_default(gmo_infile, parse_dates=True, index_col=0, sheet_name=gmo_sheets[2])
risk_free = risk_free/12

spy = pmh.read_excel_default(gmo_infile, parse_dates=True, index_col=0, sheet_name=gmo_sheets[3])
spy = spy[['SPY']]
spy['SPY'] -= risk_free['TBill 3M']

balance_sheet_columns = ['Total Assets', 'Total Capital', 'Leverage Ratio', 'Capital Ration']
ltcm_balance.columns = balance_sheet_columns
ltcm_balance.index.name = "Month"
fund_performance_columns = ['Fund Capital', 'Gross Monthly Performance', 'Net Monthly Performance', 'Index of Net Performance']
ltcm_performance.columns = fund_performance_columns
ltcm_performance.index.name = "Month"

ltcm_performance.index = risk_free.loc['1994-03':'1998-07'].index
ltcm_balance.index = risk_free.loc['1994-06':'1998-07'].index

adjust = ['Gross Monthly Performance', 'Net Monthly Performance']
ltcm_performance[adjust] = ltcm_performance[adjust].subtract(risk_free['TBill 3M'], axis=0)
ltcm_performance = ltcm_performance.join(spy, how='inner')



In [230]:
pmh.calc_summary_statistics(ltcm_performance[['Gross Monthly Performance', 'Net Monthly Performance', 'SPY']], 
                            provided_excess_returns=True, 
                            annual_factor=12, 
                            var_quantile=0.05,
                            keep_columns=['Annualized Mean', 'Annualized Vol', 'Annualized Sharpe','Skewness', 'Kurtosis', 'Annualized Historical VaR'])

,Annualized Mean,Annualized Vol,Annualized Sharpe,Skewness,Excess Kurtosis,Annualized Historical VaR (5.00%)
Gross Monthly Performance,0.2433,0.1363,1.7858,-0.2897,1.5800,-0.1052
Net Monthly Performance,0.1566,0.1118,1.4009,-0.8117,2.9172,-0.0914
SPY,0.1752,0.1127,1.5551,-0.4325,-0.3587,-0.1616


2.1.C) The gross monthly performance has a better mean return than SPY, but with increaed volatility leading to a lower sharpe ratio. The net monthly 
performance has a (almost double) negative skewness, when compared to SPY indicating siginificant tendency for negative returns. The kurtosis levels of
both the gross and net monthly performance is significantly larger than SPY pointing to a strong likelihood of both negative and positive extreme returns. Despite the more negative skewness, the VaR of the ltcm fund is more favourable than SPY. 


In [231]:
pmh.calc_regression(ltcm_performance[['Net Monthly Performance']], ltcm_performance[['SPY']], annual_factor=12, intercept=True, 
                    warnings=False, 
                    keep_columns=['R-Squared', 'Annualized Alpha', 'SPY Beta'])

,Annualized Alpha,R-Squared,SPY Beta
Net Monthly Performance,0.1326,0.0191,0.1371


2.2.B) No LTCM does not seem to be a close indexer as evidenced by the low R-squared value. This indicates that the majority of variance of the returns of LTCM cannot be explained by the market - this is consistent with their investment approach.

2.2.C) Yes, the annualized alpha of 13.26% indicates that LTCM was able to deliver substantial returns beyond the risk premium expected from market
exposure. 

In [232]:
spy['SPY Squared'] = spy['SPY']**2

pmh.calc_regression(ltcm_performance[['Net Monthly Performance']], spy[['SPY', 'SPY Squared']], annual_factor=12, intercept=True,
                    warnings=False, 
                    keep_columns=['R-Squared', 'Annualized Alpha', 'SPY Beta', 'SPY Squared Beta'])

y has lenght 53 and X has lenght 381. Joining y and X by index...


,Annualized Alpha,R-Squared,SPY Beta,SPY Squared Beta
Net Monthly Performance,0.1563,0.0245,0.1673,-1.9289


2.3.B) No, the addition of the quadratic market factor does not increase the explanatory power of the regression with the R-squared value being a 
measly 2.45%. 

2.3.C) The quadratic market factor explains the non-linear relationship between the portfolio and the market such as curvature and volatility. Given that LTCM has a negative sensitivity to this factor, all else constant, it would suggest that the portfolio is short market options.

2.3.D) Short market volatility

In [233]:
k1 = 0.03
k2 = -0.03 

spy['K1'] = np.maximum(spy['SPY'] - k1, 0)
spy['K2'] = np.maximum(spy['SPY'] - k2, 0)

pmh.calc_regression(ltcm_performance[['Net Monthly Performance']], spy[['SPY', 'K1', 'K2']], annual_factor=12, intercept=True,
                    warnings=False, 
                    keep_columns=['R-Squared', 'Annualized Alpha', 'SPY Beta', 'K1 Beta', 'K2 Beta'])



y has lenght 53 and X has lenght 381. Joining y and X by index...


,Annualized Alpha,R-Squared,SPY Beta,K1 Beta,K2 Beta
Net Monthly Performance,-0.3678,0.0566,-0.8306,-0.7897,1.3034


2.4.B) LTCM is short the call-like factor and long the put-like factor. 

2.4.C) The absolute maginture of the beta coefficients is greater for the put-like factor which indicates that the put-like factor influences the
portfolios performance more. 

2.4.D) We have established that LTCM is short the call-like factor and long the put-like factor. The difference in magnitude between betas on the option-like factors suggest that the volatility sensitivity stems from being long the market downside. 

In [234]:
infile = "../data/fx_rf_data.xlsx"

check = pd.read_excel(infile, sheet_name=None)
sheets = list(check.keys())

fx_rf = pmh.read_excel_default(infile, parse_dates=True, index_col=0, sheet_name=sheets[0])/252
fx_rates = pmh.read_excel_default(infile, parse_dates=True, index_col=0, sheet_name=sheets[1])


In [241]:
log_rf = np.log(1 + fx_rf)
log_fx = np.log(fx_rates)
log_fx_shifted = log_fx.shift(-1)

log_returns = log_fx_shifted - log_fx + log_rf 
log_returns = log_returns.subtract(log_rf['USD'], axis=0)
log_returns = log_returns.drop(['USD'], axis=1)
log_returns = log_returns.dropna()
pmh.calc_summary_statistics(log_returns, provided_excess_returns=True, 
                            annual_factor=252,
                            keep_columns=['Annualized Mean', 'Annualized Vol', 'Annualized Sharpe']).sort_values('Annualized Sharpe', ascending=False)


,Annualized Mean,Annualized Vol,Annualized Sharpe
MXN,0.0131,0.1300,0.1007
CHF,0.0031,0.1069,0.0292
GBP,-0.0243,0.0957,-0.2544
EUR,-0.0231,0.0900,-0.2568
JPY,-0.0292,0.0991,-0.2942


3.1) The difference are vast. The annualizes mean returns range from -2.92% -> 1.31 % with sharpe ratios ranging -29.42% -> 10.07%

3.2.1) UIP says that the expectation of ln(St+1) - ln(St) = ln(R*) - ln(R$), or equivalently, 0 = ln(St+1) - ln(St) + ln(R*) - ln(R$). However, as we can see from the summary statistics, all of them violate UIP as the annualized mean does not = 0. 

3.2.2) MXN had the best sharpe ratio

3.2.3) Yes, long positions of all of GBP, EUR and JPY had negative excess returns over USD.


In [ ]:
spot_differences = log_fx.shift(-1) - log_fx
rate_difference = pd.DataFrame(index=log_rf.index, columns=log_fx.columns)

for col in rate_difference.columns: 
    rate_difference[col] = log_rf['USD'] - log_rf[col]


results = {}

for col in rate_difference.columns: 
    interim = pmh.calc_regression(spot_differences[[col]], rate_difference[[col]], annual_factor=252, intercept=True, warnings=False)
    alpha = interim['Annualized Alpha'].values[0]
    beta = interim[f"{col} Beta"].values[0]
    r_squared = interim['R-Squared'].values[0]
    results[col] = [alpha, beta, r_squared]

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Alpha', 'Beta', 'R-Squared']).T
results_df

,JPY,EUR,GBP,MXN,CHF
Alpha,-0.0219,-0.0315,-0.0252,-0.0736,0.0037
Beta,0.3518,2.1602,6.8103,-0.8088,0.9069
R-Squared,0.0000,0.0003,0.0015,0.0000,0.0001


3.3.1) If the foreign rate increases then the rate differential decreases. For USD to appreiciate, we would expect the FX rate to increaese, therefore, currencies with  negatiive beta values in this regression indicate an appreciation of USD (MXN)

3.3.2) JPY, EUR, GBP, CHF

3.3.3) This predictability is strongest for GBP, as evidenced by the greatest R-Squared value

In [274]:
risk_premia = pd.DataFrame(index=rate_difference.index, columns=rate_difference.columns)
for k, col in enumerate(rate_difference.columns): 
    for i, row in enumerate(rate_difference[col]): 
        expected_return = results_df[col]['Alpha'] + (results_df[col]['Beta'] - 1) * row
        risk_premia.iloc[i, k] = expected_return

monthly_risk_premia = risk_premia.resample('M').sum()

count = {}

for col in monthly_risk_premia.columns: 
    count[col] = (monthly_risk_premia[col] > 0).sum()

count_df = pd.DataFrame.from_dict(count, orient='index', columns=['Number of Months with Positive Risk Premia'])
count_df

,Number of Months with Positive Risk Premia
JPY,0
EUR,0
GBP,0
MXN,0
CHF,202


3.4.2) All of JPY, EUR, GBP and MXN never have a positive risk premia while CHF has the most frequent positive months. 

3.4.3) Problem 1 assumed that UIP held, although we established that it did not. This approach is a data driven approach not reliant on any assumptions and could therefore provide enhanced insights.